In [1]:
import pandas as pd
from collections import Counter
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r"C:\Users\T14s\Desktop\offers\455. Finbud Daily(14-15Nov2023)\cibil-data\PII\NB86375555_AUTO-AT033-14112023-122100_14-11-2023-12-57-12_PS_QUANTIFI_V1_CV_CIR_ID.csv")
print(df.columns)

Index(['EnquiryControlNumber', 'MemberReference', 'EnquiryMemberID',
       'EnquiryMemberShortname', 'DateProcessed', 'TimeProcessed', 'name',
       'DateofBirth', 'gender', 'ID_Type', 'ID_Number', 'Telephone',
       'Tel_Extn', 'Tel_Type', 'email_id', 'address', 'StateCode', 'PINCode',
       'DateReported_address'],
      dtype='object')


In [80]:
list(df['dec_reason'].unique())

['NOT DECLINED',
 '1+ Trade60+ in 2Y',
 '1+ Trade30+ in 6M',
 'FOIR>70% with New EMI',
 '10+ Inq in lst 6Mth',
 'naps_below_thresholds',
 '2+ Trades opened in last 3Month',
 '90+% Bank Card Utilization',
 '1.No Disposable Income',
 '1+ Trade 1DPD+ in 3M',
 'SYSTEM_LOAN_OFFER<51000',
 '90+% Bank Card Utilization in last 12 month',
 'CIBIL_LT_700',
 '3.No Disposable Income',
 '1+ Trade 60+ ever',
 'High Credit Sum with 2+ Inq in 3M',
 'age<21',
 'NOBC with 8+ Inq in 12M',
 'NOBC with Cibil LT 750',
 'FOIR>70%',
 'Income<25K',
 'Months on Bureau<12',
 'Transgender',
 'NOBC with 3+ Inq in 6M',
 'PAN Error']

In [81]:
df = pd.read_csv(r"C:\Users\T14s\Desktop\offers\383. Quikr Regular DS45&46\Output\383.Quikr_Regular_DS_45&46-40923.csv",nrows=50000,low_memory=False)
# x = df[df['dec_reason'].isin(['FOIR>70% with New EMI','No Disposable Income','1.No Disposable Income','3.No Disposable Income', 'FOIR>70%'])]

In [82]:
x.shape

(10406, 224)

In [84]:
# sample1
def func1(df):

    def calci(temp,tenor=None):
        temp['max_tenor_mod'] = np.select([temp['max_tenor']==12,temp['max_tenor']==18],[18,24],36) if tenor is None else tenor
        temp['amort'] = (1 +  temp['roi_m'])**temp['max_tenor_mod']
        temp['SYSTEM_MAX_EMI'] = (temp['SYSTEM_LOAN_OFFER']*(temp['roi_m']*temp['amort'])/(temp['amort']-1)).astype(float).round(2)
        temp['dec_reason'] = np.select(condlist=[(temp['SYSTEM_MAX_EMI']>0)&(temp['SYSTEM_MAX_EMI']<= 3000)],
                                                choicelist=['FOIR>70% with New EMI'],
                                                default = "NOT DECLINED")
        return temp
    original = df[df['dec_reason']=='NOT DECLINED']
    data = df[df['dec_reason']!='NOT DECLINED']
    temp = data[(data['SYSTEM_MAX_EMI']<3000)&(data['max_tenor']<36)&(data['SYSTEM_LOAN_OFFER']>=50000)]
    temp = calci(temp)
    
    idx = temp[temp['dec_reason']=='NOT DECLINED'].index
    data.loc[idx,['max_tenor','SYSTEM_MAX_EMI','dec_reason']] = temp.loc[idx,['max_tenor','SYSTEM_MAX_EMI','dec_reason']]

    temp = data[(data['SYSTEM_MAX_EMI']<3000)&(data['max_tenor']==36)&(data['SYSTEM_LOAN_OFFER']>=50000)]
    for tenor in [24,18,12]:
        sample = temp.copy()
        sample = calci(sample,tenor)
        sample = sample[sample['dec_reason']!='NOT DECLINED']
        idx = sample[sample['dec_reason']=='NOT DECLINED'].index
        data.loc[idx,['max_tenor','SYSTEM_MAX_EMI','dec_reason']] = sample.loc[idx,['max_tenor','SYSTEM_MAX_EMI','dec_reason']]
    data = pd.concat([original,data])
    return data

In [85]:
sample = func1(x)
sample.head()

,Sequence No,Account No,Adviser Transaction No,Member_Reference_Partner,Total Features,AGG911,RVLR01,BCPMTSTR,CV11,CV14,...,l4temp,l4,ImputedInc,lending partner,hashed_phone,data_source,offer_source,randNumCol,Platform_Partner_Name,Data_Tag
4,8,NaN,2275968356,134326341,59,25.40,0.0,INDTRM,0,17,...,65000,65000,364790.538,PayU,NaN,Quikr_Regular_DS_45&46,383.Quikr_Regular_DS_45&46-40923,5370,Quikr,QKR_DS45&46__03_SEP_2023
14,19,NaN,2275968367,134326568,59,-1.00,-1.0,NOBC,0,0,...,10000,10000,40000.000,PayU,NaN,Quikr_Regular_DS_45&46,383.Quikr_Regular_DS_45&46-40923,2920,Quikr,QKR_DS45&46__03_SEP_2023
20,25,NaN,2275968373,134326824,59,5.05,0.0,RVLRPLUS,0,15,...,65000,65000,150000.000,PayU,NaN,Quikr_Regular_DS_45&46,383.Quikr_Regular_DS_45&46-40923,9536,Quikr,QKR_DS45&46__03_SEP_2023
30,38,NaN,2275968386,134759525,59,43.46,0.0,TRANSACTOR,0,7,...,65000,65000,148000.000,PayU,NaN,Quikr_Regular_DS_45&46,383.Quikr_Regular_DS_45&46-40923,323,Quikr,QKR_DS45&46__03_SEP_2023
32,42,NaN,2275968390,134520781,59,65.22,0.0,TRANPLUS,0,39,...,-21425000,-21425000,400000.000,PayU,NaN,Quikr_Regular_DS_45&46,383.Quikr_Regular_DS_45&46-40923,2574,Quikr,QKR_DS45&46__03_SEP_2023


In [86]:
sample['dec_reason'].value_counts()

FOIR>70% with New EMI     9858
3.No Disposable Income     302
1.No Disposable Income     142
FOIR>70%                   104
Name: dec_reason, dtype: int64

In [57]:
Counter(sample1['dec_reason']),Counter(sample2['dec_reason'])

(Counter({'FOIR>70% with New EMI': 6178}), Counter({'NOT DECLINED': 109652}))

In [20]:
x[['SYSTEM_LOAN_OFFER','SYSTEM_MAX_EMI','max_tenor','max_tenor_mod']]

,SYSTEM_LOAN_OFFER,SYSTEM_MAX_EMI,max_tenor,max_tenor_mod
37,51000,1998.20,24,36
74,51000,1998.20,24,36
112,50000,1855.63,24,36
169,50000,1855.63,24,36
458,51000,1998.20,24,36
...,...,...,...,...
99827,51000,1998.20,24,36
99849,51000,1998.20,24,36
99893,51000,1998.20,24,36
99901,51000,1998.20,24,36


In [56]:
df = pd.read_csv(r"C:\Users\T14s\Desktop\offers\315.Snapdeal_DS17 - Copy\Output\315.Snapdeal_DS17070723_consolidated(marketing).csv")

In [18]:
from Modules.dedupe_check import dedupe_checker

In [19]:
df['Mobile_No'] = df['Mobile_No'].astype(float).astype('int64').astype(str).str[-10:]

In [20]:
dfx = dedupe_checker(df)

Hope you updated User file, switching to partner mode


In [21]:
dfx.shape

(221972, 36)

In [22]:
dfx.to_csv('offers_generated_sd.csv',index=False)

In [23]:
df.shape

(489150, 36)

In [38]:
dfx.shape

(221972, 36)

In [28]:
save_path= r"C:\Users\T14s\Desktop\offers\315.Snapdeal_DS17 - Copy\Output"
total_data_count = 2900769
x=pd.pivot_table(dfx, values='Mobile_No', index=['Category'],
                    columns=['propensity'], aggfunc=['count'],fill_value=0,margins = True, margins_name='Total')
x['percentage']=((x[('count', 'Total')])*100/total_data_count).round(2)
x['percentage']=(x['percentage'].astype(str))+'%'
x.loc['percentage'] = [(((x[('count', 'HiiProp')][-1]/total_data_count)*100).round(2).astype(str))+'%',
                                (((x[('count', 'LowProp')][-1]/total_data_count)*100).round(2).astype(str))+'%',
                                (((x[('count', 'MedProp')][-1]/total_data_count)*100).round(2).astype(str))+'%',
                                (((x[('count', 'Total')][-1]/total_data_count)*100).round(2).astype(str))+'%','']
x.to_csv(save_path+'\Cat-prop_pivot_old .csv')

In [51]:
def pivotCreator(mfx):
    # pivot Table file creation
    mfx.rename(columns={"pred_propensity":"PROPENPLSC Score Value"},inplace=True)
    df_pivot = mfx[mfx["dec_reason"] == "NOT DECLINED"].copy()
    req_stats = {
        "Member Reference": ["count"],
        "BC02S": ["mean"],
        "BC28S": ["mean"],
        "New_FOIR": ["mean"],
        "CV14_1M": ["max", "sum", "mean"],
        "CV14_3M": ["max", "sum", "mean"],
        "TRD": ["mean"],
        "CIBILTUSC3 Score Value": ["min", "mean"],
        "PROPENPLSC Score Value": ["min", "mean"],
        "ImputedInc": ["mean"],
        "SYSTEM_PF": ["mean"],
        "SYSTEM_RATEOFINT": ["mean"],
        "SYSTEM_LOAN_OFFER": ["min", "mean", "max"],
        "SYSTEM_MAX_EMI": ["mean"],
        "max_tenor": ["mean"],
        "naps_tu": ["min", "mean", "max"],
        
    }
    req_cols = list(req_stats.keys()) + [
        "riskband",
        "riskband2",
        "lending partner",
    ]
    df_pivot = df_pivot[req_cols]

    pivot = None
    for lender in ["Muthoot", "PayU", "Liquiloans"]:
        Pivot = (
            df_pivot[df_pivot["lending partner"] == lender]
            .groupby(["riskband", "riskband2"])
            .agg(req_stats)
        )
        Pivot['lender']=lender
        pivot = pd.concat([pivot, Pivot])

    pivot = pd.concat(
        [pivot, df_pivot.groupby(["riskband", "riskband2"]).agg(req_stats)]
    )
    pivot.to_excel(save_path / "pivot.xlsx")



In [34]:
mf = pd.read_csv(r"C:\Users\T14s\Desktop\offers\315.Snapdeal_DS17 - Copy\Output\315.Snapdeal_DS17070723.csv")

In [48]:
mfx = mf[mf['Member Reference'].isin(dfx['Mobile_No'])]

In [49]:
mfx.shape

(222059, 200)

In [44]:
dfx['Mobile_No']

2         6201096927
3         6201292282
4         6201848401
9         6238117235
19        6261406108
             ...    
489137    9999350395
489138    9999376506
489140    9999494525
489145    9999651913
489149    9999995457
Name: Mobile_No, Length: 221972, dtype: object

In [46]:
mf['Member Reference'] = mf['Member Reference'].astype(float).astype('int64').astype(str).str[-10:]

In [52]:
pivotCreator(mfx)

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [54]:
    df_pivot = mfx[mfx['dec_reason'] == 'NOT DECLINED'].copy()

    df_pivot = df_pivot[["riskband",
                         "riskband2",
                        "lending partner",
                         "Member Reference",
                         "BC02S",
                         "BC28S",
                         "New_FOIR",
                         "CV14_1M",
                         "CV14_3M",
                         "TRD",
                         "CIBILTUSC3 Score Value",
                         "PROPENPLSC Score Value",
                         "ImputedInc",
                         "SYSTEM_PF",
                         "SYSTEM_RATEOFINT",
                         "SYSTEM_LOAN_OFFER",
                         "SYSTEM_MAX_EMI",
                         "max_tenor"]]

    Pivot_Muthoot = df_pivot[df_pivot["lending partner"] == "Muthoot"].groupby(["riskband", "riskband2"]).agg({"Member Reference": ["count"],
                                                                                                               "BC02S": ["mean"],
                                                                                                              "BC28S": ["mean"],
                                                                                                               "New_FOIR": ["mean"],
                                                                                                               "CV14_1M": ["max", "sum", "mean"],
                                                                                                               "CV14_3M": ["max", "sum", "mean"],
                                                                                                               "TRD": ["mean"],
                                                                                                               "CIBILTUSC3 Score Value": ["min", "mean"],
                                                                                                               "PROPENPLSC Score Value": ["min", "mean"],
                                                                                                               "ImputedInc": ["mean"],
                                                                                                               "SYSTEM_PF": ["mean"],
                                                                                                               "SYSTEM_RATEOFINT": ["mean"],
                                                                                                               "SYSTEM_LOAN_OFFER": ["min", "mean", "max"],
                                                                                                               "SYSTEM_MAX_EMI": ["mean"],
                                                                                                               "max_tenor": ["mean"]})

    Pivot_PayU = df_pivot[df_pivot["lending partner"] == "PayU"].groupby(["riskband", "riskband2"]).agg({"Member Reference": ["count"],
                                                                                                        "BC02S": ["mean"],
                                                                                                         "BC28S": ["mean"],
                                                                                                         "New_FOIR": ["mean"],
                                                                                                         "CV14_1M": ["max", "sum", "mean"],
                                                                                                         "CV14_3M": ["max", "sum", "mean"],
                                                                                                         "TRD": ["mean"],
                                                                                                         "CIBILTUSC3 Score Value": ["min", "mean"],
                                                                                                         "PROPENPLSC Score Value": ["min", "mean"],
                                                                                                         "ImputedInc": ["mean"],
                                                                                                         "SYSTEM_PF": ["mean"],
                                                                                                         "SYSTEM_RATEOFINT": ["mean"],
                                                                                                         "SYSTEM_LOAN_OFFER": ["min", "mean", "max"],
                                                                                                         "SYSTEM_MAX_EMI": ["mean"],
                                                                                                         "max_tenor": ["mean"]})

    Pivot_Liquiloans = df_pivot[df_pivot["lending partner"] == "Liquiloans"].groupby(["riskband", "riskband2"]).agg({"Member Reference": ["count"],
                                                                                                                     "BC02S": ["mean"],
                                                                                                                    "BC28S": ["mean"],
                                                                                                                     "New_FOIR": ["mean"],
                                                                                                                     "CV14_1M": ["max", "sum", "mean"],
                                                                                                                     "CV14_3M": ["max", "sum", "mean"],
                                                                                                                     "TRD": ["mean"],
                                                                                                                     "CIBILTUSC3 Score Value": ["min", "mean"],
                                                                                                                     "PROPENPLSC Score Value": ["min", "mean"],
                                                                                                                     "ImputedInc": ["mean"],
                                                                                                                     "SYSTEM_PF": ["mean"],
                                                                                                                     "SYSTEM_RATEOFINT": ["mean"],
                                                                                                                     "SYSTEM_LOAN_OFFER": ["min", "mean", "max"],
                                                                                                                     "SYSTEM_MAX_EMI": ["mean"],
                                                                                                                     "max_tenor": ["mean"]})

    Pivot_All = df_pivot.groupby(["riskband", "riskband2"]).agg({"Member Reference": ["count"],
                                                                "BC02S": ["mean"],
                                                                 "BC28S": ["mean"],
                                                                 "New_FOIR": ["mean"],
                                                                 "CV14_1M": ["max", "sum", "mean"],
                                                                 "CV14_3M": ["max", "sum", "mean"],
                                                                 "TRD": ["mean"],
                                                                 "CIBILTUSC3 Score Value": ["min", "mean"],
                                                                 "PROPENPLSC Score Value": ["min", "mean"],
                                                                 "ImputedInc": ["mean"],
                                                                 "SYSTEM_PF": ["mean"],
                                                                 "SYSTEM_RATEOFINT": ["mean"],
                                                                 "SYSTEM_LOAN_OFFER": ["min", "mean", "max"],
                                                                 "SYSTEM_MAX_EMI": ["mean"],
                                                                 "max_tenor": ["mean"]})

    Pivot_Muthoot['Lender'] = 'Muthoot'
    Pivot_PayU['Lender'] = 'PayU'
    Pivot_Liquiloans['Lender'] = 'Liquiloans'
    Pivot_All['Lender'] = 'PayU & Liquiloans'
    pivot = pd.concat([Pivot_Muthoot, Pivot_PayU, Pivot_Liquiloans, Pivot_All], axis=0)
    pivot.to_excel(save_path +'\pivot1.xlsx')


In [57]:
dfx = pd.read_csv(r"C:\Users\T14s\Desktop\offers\315.Snapdeal_DS17 - Copy\offers_generated_sd.csv")

In [58]:
dfx.shape

(221972, 36)

In [ ]:
df